<a href="https://colab.research.google.com/github/UPstartDeveloper/Barrier_Reef_Protector/blob/main/Protect_GBR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Starter Code from Kaggle Notebook
This is just being stored here for historical reasons. Will remove later.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import sys

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, RUN THIS (by clicking run or pressing Shift+Enter) to list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Double-check Versions

In [2]:
print(sys.version)  # Python version
print(tf.__version__) # on Kaggle, this will be 2.6

3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
2.7.0


### Skipping to Modelling For a Second

1. Colab-specific: Mount Google Drive (we can do this by clicking the appropiate icons on the screen, please see here for more info)
2. Let's just test if we can load in YOLOv5 from ultralytics
2. then export to a SavedModel
3. then load the model back in, using TF 2

In [3]:
# Colab-specific - find a folder to store YOLOv5 stuff
%cd ./drive/MyDrive

/content/drive/MyDrive


**Step 2:** Now let's load the YOLOv5 model from the PyTorch Hub:

In [6]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 10064, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 10064 (delta 11), reused 16 (delta 8), pack-reused 10043
Receiving objects: 100% (10064/10064), 10.36 MiB | 8.65 MiB/s, done.
Resolving deltas: 100% (6981/6981), done.
/content/drive/MyDrive/yolov5


In [7]:
from yolov5 import utils
display = utils.notebook_init()  # running the checks

YOLOv5 🚀 2021-11-28 torch 1.10.0+cu111 CPU


Setup complete ✅


**Step 3:** Now, let export to a `SavedModel` (this is using the "nano" version of YOLOv5, the smallest):

In [9]:
!python export.py --weights yolov5n.pt --include saved_model

export: data=data/coco128.yaml, weights=yolov5n.pt, imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, train=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=13, verbose=False, workspace=4, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['saved_model']
YOLOv5 🚀 2021-11-28 torch 1.10.0+cu111 CPU

100% 3.77M/3.77M [00:00<00:00, 22.7MB/s]

Fusing layers... 
Model Summary: 213 layers, 1867405 parameters, 0 gradients

PyTorch: starting from yolov5n.pt (4.0 MB)

TensorFlow saved_model: starting export with tensorflow 2.7.0...

                 from  n    params  module                                  arguments                     
2021-11-28 22:37:27.223108: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
  0                -1  1      1760  models.common.Conv                      [3, 16, 6, 2, 2]              
  1                -1  1  

**Step 4:** Now, let's test loading the model back in via Tensorflow 2:

In [10]:
tf_model = tf.keras.models.load_model('./yolov5n_saved_model')

No training configuration found in save file, so the model was *not* compiled. Compile it manually.


Woohoo! We got a warning that the model has no training configuration, but that is to be expected (since we want to do all the training code henceforth ourselves, using Tensorflow code). 

Let's get started!